In [ ]:
from selenium import webdriver
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

In [ ]:
def get_url(location):
    """Navigates to the rental listing page for the specified location and returns the WebDriver instance."""
    url = "https://bostonpads.com/"
    driver = webdriver.Chrome()
    
    # Visit the main website
    driver.get(url)

    # Open the location dropdown menu
    dropdown_element = driver.find_element(By.XPATH, '//div[@class="searchable_location"]/input[@id="primary_loc"]')
    dropdown_element.click()
    
    # Select the desired location from the dropdown
    location_element = driver.find_element(By.XPATH, f'//div[@class="searchable_location"]/ul/li[contains(text(), "{location}")]')
    location_element.click()

    # Click the search button to go to the rental listing page
    search_button = driver.find_element(By.XPATH, '//button[@name="button"]')
    search_button.click()

    return driver

In [ ]:
def extract_details(item):
    """Extracts relevant property details from a BeautifulSoup element and returns them as a tuple."""

    # Get the location
    location = item.find('div', class_='listingLocation')
    location = location.text.strip() if location else None

    # Get the rent
    rent = item.find('div', class_='rentPrice')
    rent = rent.text.strip() if rent else None

    # Get the number of bedrooms and bathrooms
    beds_baths = item.find('div', class_='listingBedBath')
    if beds_baths:
        # Get the number of bedrooms
        beds = beds_baths.find(class_='listingBed')
        beds = beds.text.strip() if beds else None
        
        # Get the number of bathrooms
        baths = beds_baths.find(class_='listingBath')
        baths = baths.text.strip() if baths else None
    else:
        beds, baths = None, None

    # Get the amenities
    amenities = item.find('div', class_='listingAmenities')
    amenities = amenities.text.strip() if amenities else None

    # Get the available date
    available_date = item.find('div', class_='listingAvailable')
    available_date = available_date.text.strip() if available_date else None

    # Get the property URL
    url = item.find('a', class_='listingImage')
    url = url['href'] if url else None

    return (location, rent, beds, baths, amenities, available_date, url)

In [ ]:
def scrape_properties(location):
    """Scrapes rental property listings for the specified location and saves the results as a CSV file."""
    records = []
    
    # Get the WebDriver instance with the rental listing page loaded
    driver = get_url(location)

    # Loop through pages 1 to 10
    for page_number in range(1, 11):
        time.sleep(5)

        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Find all property listings on the page
        results = soup.find_all('div', class_='listingBox')

        # Extract property details from each listing and add them to the records list
        for item in results:
            record = extract_details(item)
            if record:
                records.append(record)

        if page_number != 10:
            next_page_element = driver.find_element(By.XPATH, f'//li[@class=" pagination-link" and @data-page="{page_number + 1}"]/a')
            next_page_element.click()

    driver.close()

    # Convert the records list to a DataFrame
    df = pd.DataFrame(records, columns=['Location', 'Rent', 'Beds', 'Baths', 'Amenities', 'Available Date', 'URL'])
    print(df)

    # Save the records to a CSV file
    filename = "bostonpads.csv"
    with open(filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Location', 'Rent', 'Beds', 'Baths', 'Amenities', 'Available Date', 'URL'])
        writer.writerows(records)

In [5]:
scrape_properties('Boston')

                                   Location     Rent    Beds      Baths  \
0                              Alyx Seaport  $8,372*  3 Beds    2 Baths   
1                               The Sudbury  $4,142*   1 Bed     1 Bath   
2                               The Sudbury  $5,456*  2 Beds    2 Baths   
3                                      Velo  $2,866*   1 Bed     1 Bath   
4       Savin Hill - Dorchester, Boston, MA   $2,800  3 Beds     1 Bath   
..                                      ...      ...     ...        ...   
395   Boston College - Brighton, Boston, MA   $5,700  4 Beds    2 Baths   
396   Boston College - Brighton, Boston, MA   $5,200  5 Beds    2 Baths   
397  Thomas Park - South Boston, Boston, MA   $6,800  4 Beds  4.5 Baths   
398              Fenway/Kenmore, Boston, MA   $3,450  2 Beds     1 Bath   
399                     Allston, Boston, MA   $2,695   1 Bed     1 Bath   

                                             Amenities         Available Date  \
0    Pet Friendly,